In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist


In [2]:
df_business = pd.read_csv('restaurantes_california.csv')
df_business

,Unnamed: 0,gmap_id,name,latitude,longitude,combined_categories,num_of_reviews,avg_rating,cluster,primary_category
0,1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,34.058092,-118.292130,Korean restaurant,18,4.4,14,restaurant
1,5,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,33.916402,-118.010855,Restaurant,18,4.5,14,restaurant
2,46,0x808f879f35b5088b:0xe3541cec7a95bd88,TACOS LA CABANA,37.789076,-122.233884,Taco restaurant,2,5.0,14,restaurant
3,48,0x808f87f90c1f661f:0xf384e804a61e0c0b,Mariscos el poblano,37.764203,-122.214647,Restaurant,3,5.0,14,restaurant
4,59,0x80dcd95d192d988b:0x68795f58e35bf888,Off The Hoof,33.748329,-117.866045,Restaurant,3,4.0,14,restaurant
...,...,...,...,...,...,...,...,...,...,...
23905,299713,grBPIq_eJCT_SGuhvMrUZQ,Blaze Pizza,34.440160,-119.752421,"Fast Food, Restaurants, Pizza, Salad",146,3.5,12,restaurant
23906,300342,XQY1_EorK7FMCffiftN0fA,Papa John's Pizza,34.441749,-119.823092,"Restaurants, Food, Food Delivery Services, Pizza",52,2.5,12,restaurant
23907,300449,v5jxpGnKbx3vVxs28dgokQ,Saigon Vietnamese Restaurant,34.439514,-119.751631,"Restaurants, Vietnamese, Vegetarian",192,3.0,4,restaurant
23908,300630,TCWnTa69sJ3vGQsdEe3Esw,Uncle Roccos Famous NY Pizza,34.416585,-119.695533,"Pizza, Italian, Fast Food, Restaurants",317,2.5,12,restaurant


In [11]:
# agrupo datos
encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(df_business[['combined_categories']])


In [12]:

features = np.hstack([encoded_categories.toarray(), df_business[['avg_rating']].values])


In [13]:
#calculo similitud coseno
similarity_matrix = cosine_similarity(features)


In [14]:
# función recomendacion
def recommend_similar_businesses(business_index, top_n=5):
    # obtener similitudes para el negocio dado
    similarity_scores = list(enumerate(similarity_matrix[business_index]))
    
    # ordeno negocios por similitud y filtrar el mismo negocio
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:] 
    
    # obtener índices de los negocios más similares
    similar_indices = [i[0] for i in similarity_scores[:top_n]]
    
  
    return df_business.iloc[similar_indices]


In [16]:
business_index = 0  # pruebo indice 0 restaurante coreano
recommended_businesses = recommend_similar_businesses(business_index)
recommended_businesses

,Unnamed: 0,gmap_id,name,latitude,longitude,combined_categories,num_of_reviews,avg_rating,cluster,primary_category
1160,24685,0x80c2b99515af5bcb:0xe01e1cc7505ebe62,Jun Won Restaurant,34.066697,-118.308956,Korean restaurant,107,4.4,14,restaurant
3571,45729,0x80dd2c3ed3abd951:0xd45fe94d4e70b041,Jangmo Jip Restaurant,33.846876,-118.047760,Korean restaurant,188,4.4,14,restaurant
5815,63072,0x80c2b888676aeb11:0x76290c92bdf0b15d,Yeosu,34.052926,-118.312503,Korean restaurant,18,4.4,14,restaurant
8906,97638,0x80c2c78a4f0f3f59:0x1000d280ef66ef5c,Jokbal 1030,34.064028,-118.296808,Korean restaurant,34,4.4,14,restaurant
10149,123141,0x80856691820646a5:0x2c38c1f098c31a0b,Keunjib Restaurant DINE IN,37.982301,-122.067057,Korean restaurant,85,4.4,14,restaurant


In [19]:
df_business.head()

,Unnamed: 0,gmap_id,name,latitude,longitude,combined_categories,num_of_reviews,avg_rating,cluster,primary_category
0,1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,34.058092,-118.292130,Korean restaurant,18,4.4,14,restaurant
1,5,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,33.916402,-118.010855,Restaurant,18,4.5,14,restaurant
2,46,0x808f879f35b5088b:0xe3541cec7a95bd88,TACOS LA CABANA,37.789076,-122.233884,Taco restaurant,2,5.0,14,restaurant
3,48,0x808f87f90c1f661f:0xf384e804a61e0c0b,Mariscos el poblano,37.764203,-122.214647,Restaurant,3,5.0,14,restaurant
4,59,0x80dcd95d192d988b:0x68795f58e35bf888,Off The Hoof,33.748329,-117.866045,Restaurant,3,4.0,14,restaurant


In [20]:
# Preprocesamiento de datos
encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(df_business[['combined_categories']])

# Concatenar características codificadas con avg_rating
features = np.hstack([encoded_categories.toarray(), df_business[['avg_rating']].values])


In [21]:
# Función para recomendar negocios similares con prioridad en avg_rating
def recommend_similar_businesses(business_index, top_n=5):
    # Obtenemos las similitudes para el negocio dado
    similarity_scores = list(enumerate(similarity_matrix[business_index]))
    
    # Ordenar los negocios por similitud
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:]  # Omitimos el primer resultado que es el propio negocio
    
    # Filtrar los negocios con mejores avg_rating
    filtered_scores = sorted(similarity_scores, key=lambda x: df_business.iloc[x[0]]['avg_rating'], reverse=True)
    
    # Obtener los índices de los negocios más similares y mejor calificados
    similar_indices = [i[0] for i in filtered_scores[:top_n]]
    
    # Retornar los negocios recomendados
    return df_business.iloc[similar_indices]


In [23]:
# Ejemplo de uso
business_index = 0  # Índice del negocio para el cual queremos hacer la recomendación
recommended_businesses = recommend_similar_businesses(business_index)
recommended_businesses

,Unnamed: 0,gmap_id,name,latitude,longitude,combined_categories,num_of_reviews,avg_rating,cluster,primary_category
780,17658,0x80c2c73584d5bd79:0xb3213967e780d72,Jook Hyang II,34.052247,-118.290223,Korean restaurant,1,5.0,14,restaurant
8859,96947,0x808fcbf2b23ed8cb:0x663b31677542c094,Seorai Korean Bbq,37.351984,-121.981579,Korean restaurant,3,5.0,14,restaurant
9044,100451,0x80dbf92c2737a335:0x83bf3c237fbc9203,Sue’s Korean Kitchen,32.906190,-117.173740,Korean restaurant,8,5.0,14,restaurant
10539,131005,0x80c2db1b6da49399:0x6a8e88ebbc1d731a,Genghis Khan,34.080185,-118.102804,Korean restaurant,2,5.0,14,restaurant
11299,144432,0x80dcdfbe9356d2ef:0x95ada76afa614787,Nabi Bento,33.699653,-117.865071,Korean restaurant,6,5.0,14,restaurant


In [26]:

def recommend_businesses(business_index, top_n=5):
    # obtener similitudes para el negocio dado
    similarity_scores = list(enumerate(similarity_matrix[business_index]))
    
    # ordeno los negocios por similitud
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:]  
    
    #filtro los negocios con mejores avg_rating
    filtered_scores = sorted(similarity_scores, key=lambda x: df_business.iloc[x[0]]['avg_rating'], reverse=True)
    
    #obtener los índices de los mejores
    similar_indices = [i[0] for i in filtered_scores[:top_n]]
    
    #retorno
    return df_business.iloc[similar_indices]['name'].tolist()


In [27]:
# Ejemplo de uso
business_index = 0  # Índice del negocio para el cual queremos hacer la recomendación
recommended_business_names = recommend_businesses(business_index)
recommended_business_names

['Jook Hyang II',
 'Seorai Korean Bbq',
 'Sue’s Korean Kitchen',
 'Genghis Khan',
 'Nabi Bento']

# VERSION FINAL

In [32]:
from scipy.spatial.distance import cdist
import numpy as np

In [28]:
#procesar de datos
encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(df_business[['combined_categories']])


In [29]:
#unir con avg_rating
features = np.hstack([encoded_categories.toarray(), df_business[['avg_rating']].values])


In [30]:
#calculo similitud de coseno
similarity_matrix = cosine_similarity(features)


In [33]:
#calcular distancia geográfica
coords = df_business[['latitude', 'longitude']].values
distance_matrix = cdist(coords, coords, metric='euclidean')


In [34]:
def recommend_businesses(business_index, top_n=5, max_distance=0.1):
    #obtener las similitudes 
    similarity_scores = list(enumerate(similarity_matrix[business_index]))
    
    #ordeno los negocios por similitud
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:]  # Omitimos el primer resultado que es el propio negocio
    
    # filtro distancia geográfica
    nearby_businesses = [i for i in similarity_scores if distance_matrix[business_index][i[0]] <= max_distance]
    
    #si no hay suficientes negocios cercanos, achicar la restricción de distancia
    if len(nearby_businesses) < top_n:
        nearby_businesses = similarity_scores
    
    #filtrar  mejores avg_rating
    filtered_scores = sorted(nearby_businesses, key=lambda x: df_business.iloc[x[0]]['avg_rating'], reverse=True)
    
    #obtengo indices
    similar_indices = [i[0] for i in filtered_scores[:top_n]]
    
    #retorno nombres de los negocios en forma de lista
    return df_business.iloc[similar_indices]['name'].tolist()


# Prueba

In [35]:

business_index = 0  # Índice 0 restaurante coreano
recommended_business_names = recommend_businesses(business_index, max_distance=0.2)
recommended_business_names

['Jook Hyang II',
 'Sulga',
 'Tacos El Comelon (Food Truck)',
 'Hanmaeum Restaurant',
 'Baon - Filipino Lunch Counter']